In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:
def create_line_iterator(P1, P2, img):
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3), dtype=np.int32)
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    return itbuffer

In [3]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub = rospy.Publisher('updated_map', Map, queue_size=10)

occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0
map_height = 0
camera_range = 85/map_resolution
overlay = False
display_robot_yaw = False
display_rect = True
display_map = True
display_info = False

np.set_printoptions(threshold=sys.maxsize)

In [4]:
def rectangle(mat,color,image,original):
    
    global bearings
    global overlay
    global occ_map
    global tags
    global display_info
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if (width < height and 4*width > height and width*height > 45):
            if(color =='green'):
                tag = 2
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            elif(color =='red'):
                tag = 3
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            print min_rect[1][0]*min_rect[1][1]
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        return image

In [5]:
def get_mapnbot_info(trans,robot_yaw):

    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    
    map_array = occ_map.data

    map_width = occ_map.info.width
    map_height = occ_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
    
    return map_np, robot_yaw

In [6]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [7]:
def map_filter(trans, robot_yaw):
    
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
    if display_robot_yaw and occ_map.header.seq % 5 == 0:
        yaw_offset.append(0.0)
        print "Robot_yaw: ", robot_yaw
        print "Buoy yaw: ", buoy_yaw
        print len(slices)
##############################################################################
    
    slices = []
    i = 0
    while (i < len(bearings)):
        buoy_yaw = robot_yaw + bearings[i]
        if ((buoy_yaw) < 0):
            buoy_yaw = buoy_yaw + 360
        point_deltax = camera_range * math.sin(math.radians(buoy_yaw))
        point_deltay = camera_range * math.cos(math.radians(buoy_yaw))
        
        point1 = np.array([round(robot_x), round(robot_y)])
        point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)])
        sliced = create_line_iterator(point1.astype(int),point2.astype(int),map_np)
        slices.append(sliced)
###############################################################################################     

# If you uncomment the code below, make sure you change tags[i//1] to tags[i//3] in the draw_ray function

#         point3 = np.array([round(robot_x+1), round(robot_y)])
#         point4 = np.array([round(robot_x+point_deltax+1), round(robot_y+point_deltay)])
#         sliced = create_line_iterator(point3.astype(int),point4.astype(int),map_np)
#         slices.append(sliced)
        
#         point5 = np.array([round(robot_x-1), round(robot_y)])
#         point6 = np.array([round(robot_x+point_deltax-1), round(robot_y+point_deltay)])
#         sliced = create_line_iterator(point5.astype(int),point6.astype(int),map_np)
#         slices.append(sliced)
################################################################################################
        i+=1
    
    return slices

In [8]:
def draw_background_objects(map_img,map_np):
    
    global origin
    global robot_x
    global robot_y
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    map_img[map_np == -1] = [255]    #Change to [255,255,255] for color
    map_img[map_np == 0] = [255]     #Change to [255,255,255] for color 
    map_img[map_np == 100] = [0]#Change to [0,0,0] for color
    origin_color = [0]#Change to [0,0,255] for color
    robot_color = [0]
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
    
    return map_img

In [9]:
def draw_ray(map_img,slices):
    
    global display_robot_yaw
    global bearings
    global tags
    
    slices_size = len(slices)
    
    if not(display_robot_yaw):
        for i in range(int(slices_size)):
            try:
                found = False
                arr = slices[i]
                for ray in arr:
                    #print slice
                    if not found and (ray[2] == -1 or ray[2] == 0):
                        map_img[ray[1]][ray[0]] = [0]#Change to [255,0,0] for color
                    elif ray[2] == 100:
                        if tags[i//1] == 2: #here
                            map_img[ray[1]][ray[0]]=[180]#Change to [0,255,0] for color
                            found = True
                        if tags[i//1] == 3: #and here
                            map_img[ray[1]][ray[0]]=[230]#Change to [0,0,255] for color
                            found = True
            except Exception as ex:
                print ex
                print slices[i]
                print
    else:
        for arr in slices[-1]:
            map_img[arr[1]][arr[0]] = [0,255,0]
    return map_img

In [10]:
def get_map(slices):
    
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))

    map_img = np.zeros((map_height, map_width, 3))
    
    map_img = draw_background_objects(map_img,map_np)
    
    map_img = draw_ray(map_img,slices)
    
    map_img = np.swapaxes(map_img, 0, 1)
    
    return map_img

In [11]:
def disp_map(map_img):
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [16]:
def transform(img):
    
    global bearings
    global tags
# Once a color image has been read, it passes through the 
# transform function. The transform function has two parts:
# pre-processing also known as pp which filters the image based
# of certain values in the hue, saturation, and value (hsv) 
# colorspace and the dbscan which clusters the processed image
# and classifies the clusters as either a buoy, a green buoy,
# a red buoy, or noise. No changes are needed.
    
    
    
    bridge = cv_bridge.CvBridge()
    
    bearings = []
    tags = []
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    
    
    image = pp(img,width,height)
    
    img2= dbscan(image, img)
    
    return img2
def pp(image, width, height):
    
# The first function in transform is pp or preprocessing. The 
# first line truncates the lower half of the image. The reason
# this is because the plantoons from the wamv are always 
# blocking the camera vision and become noisy data. The
# function changes the image to the HSV colorspace and 
# filters out the image. No changes are needed. It returns 
# the filtered image in the BGR colorspace.The input is the
# color image.

    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)

    return(image)
def dbscan(image, original):
# The second function in transform is dbscan. The inputs are 
# the filtered colored image and the original color image
# read from the imread statement. The output is the original
# image.This is the end of this classification algorithm.

###########################################################

# Part 1: Thresholding the image
##########################################################
# The reason we threshold the image before applying DBSCAN to 
# the points is because DBSCAN is a computational demanding 
# function. This part takes the filtered image, makes a 
# copy of it, and converts it into a grayscale image. The gray-
# scale image is thresholded at a value of 10. 0 is black and
# 255 is white. This takes the ~400,000 matrix points and 
# leaves only ~10,000 nonzero matrix points. 

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

# After thresholding the image, we pull out those nonzero
# points and assign it to the nonzero array variable. This
# array has dimensions of n rows and 2 columns. The first 
# column is the y coordinate and the second column is the x
# coordinate. These two arrays would be combined by 
# np.column_stack which merges the two arrays at their columns.

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
#########################################################

# Creating the BDSCAN object and Sorting Array with Labels
#########################################################    
# After thresholding the filtered image and creating an
# array of nonzero coordinates, this algorithm applies a
# clustering algorithm that filters the image based off the
# array X. If X is empty, this function ends and output is 
# the original input. The first line creates the DBSCAN
# object with an epsilon of 3 and minimum samples of 20. 
# Eps refers to the size of the neighborhood and the 
# min_samples refers to the minimum number of samples required
# for that neighborhood to be considered a cluster. The user
# is free to change the two values for eps and min_samples. The
# .fit(X) applies the matrix points to the DBSCAN object.
# One of the parameters of the db scan object are the labels.
# This is invoked by the line labels = db.labels. This creates
# an array of labels that assigns a number to the image index.
# n clusters have labels that start at 0 and end with n-1.For 
# example, if an image has four clusters, the labels would
# include 0,1,2, and 3. -1 is dedicated for noise. 

    if (len(X) > 0):
        start_time = time.time()
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_
#         print (1.0*(time.time()-start_time))

# The array variable combines the x and y coordinates and the 
# labels in a single array aligned by column. The array is
# sorted with the sort variable with the values in the third
# column or the labels. For example, an image with four
# clusters and noise would be labeled in order from (-1,0,1,2,3).
# The while loop iterates from the beginning of the sorted loop
# and deletes every instance of -1 or until the list is empty.
# This while loop is needed because the noise data is
# irrelevant to the cluster algorithm and the user does not 
# know how much noise there is in the image. The image could 
# have no noise or it could all be noise. The output of this
# section is an array with coordinates sorted together in 
# clusters.

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]
#########################################################

# Iterating through and classifying each cluster
#########################################################
# After iterating through sort, the program is either left with
# an empty array or an array of coordinates sorted and grouped
# by each cluster. The user needs the number of clusters. The 
# next three lines in the big if block does this. unique_labels
# sorts the set of labels and removes -1 if present. The next 
# two lines creates a copy of the sort array and removes the 
# labels column from the array.

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

# The program assigns three variables. The first one i iterates
# through sort and compare its label value to x for each 
# iteration until the very end of the sort array. The second
# x refers to the cluster that the program is currently class-
# ifying. It starts with 0 and continues classifying until it 
# has classified all clusters. The third variable is l and this
# array becomes all the points within a cluster. Every time the 
# variable i iterates through sort and is equal to the value of
# the current cluster x, the coordinates are appended to the 
# array l. Since this array is sorted, the first time the x 
# variable equals a different value- the cluster has been 
# accounted for. This array of l contains all the points within
# the cluster. This array and the filtered image gets passed
# through the color function. The result of the color function
# gets passed through the rectangle function with the array,
# filtered image, and original image. The x variable is 
# incremented for the next cluster and the l array is set equal 
# to zero. Once the while loop has iterated through all 
# clusters, the result of the rectangle function is set to the 
# original image and the original image is returned to the 
# transform function which is returned to the read function 
# which writes the image.

            i = 0
            x = 0
            l = []
            image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original)
                    l = []
            global overlay
            if not overlay:
                tb = cv.cvtColor(tb, cv.COLOR_HSV2BGR)
            original = tb
        print ""
    return original
def color(mat,image):
    
    ############################################################

# For the color function, the points of the cluster and the 
# filtered image are passed through. In this function, the 
# average red, green, and blue pixel values are calculated
# from the image inside of the cluster. If the average value
# fits within a certain range, the string 'red' or 'green' is 
# returned. Otherwise, the function returns the string 'none'
# The user is free to change the range of the BGR values that
# determine whether the buoy is red or green.

    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 80 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 30 and (sumred/len(mat)) < 80 and (sumgrn/len(mat)) > 60 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'
def color_hsv(mat, image):
    
# The rectangle function's parameters are the array of coordinates
# of the cluster, the color of the cluster, the filtered image,
# and the original image. The boundingRect function applies
# a non-rotated rectangle to the image points and returns four
# values in an array, the x and y coordinates of the leftmost
# corner and the width and height. These values are converted
# to ints. The font is set to Hershey Simple for the text and
# the tag is set to zero. The tag is useful in the classifica-
# tion of the buoy. The first classification of the buoy is 
# the proportion of the width and height. If the height is
# between 1.5 times the width and 3.5 times the width, the
# cluster is considered a buoy. If the cluster is given a 
# 'red' or 'green' color, it would be classified as a 'red buoy'
# or 'green buoy' respectively. Otherwise, it would simply be
# classified as a buoy. If any of these three options occur, 
# the tag is set to 1. If the height is not in that range
# mentioned above and the color is 'none', the cluster is
# classified as noise. The last function rectangle makes a 
# rectangle over the original image. The original image is sent
# back to the DBSCAN function.


    
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    #print mat
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)

    return None

In [13]:
def callback(rect):
    
    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    
    if not rospy.is_shutdown():       
        try:
            now = rospy.Time.now()
            listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(0.5))
            (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
            pixel_lines = map_filter(trans, math.degrees(euler[2]))
            map_image = get_map(pixel_lines)
#             print(map_image)
#             up_occ_map.data = map_image
            pub.publish(up_occ_map)
            if display_rect == True:
                disp_rect(rect)
            if display_map == True:
                disp_map(map_image)
        except tf.LookupException as l:
            print l
        except tf.ConnectivityException as c:
            print c
        except tf.ExtrapolationException as e:
            print e
        except Exception as ex:
            print ex 

In [14]:
def map_callback(p_map):
    
    global occ_map
    global up_occ_map
    
    occ_map = p_map    
    up_occ_map = p_map

In [15]:
if __name__ == '__main__':
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()   

48.0
242.0
176.896579473

36.0
206.292143859
176.896579473

44.0
205.101582079
176.896579473

88.1797759081
205.101582079
176.896579473

69.9999973641
213.033743649
160.0

40.5999997716
191.028925526
159.468977161

33.0
208.108107999
176.896579473

34.6153797236
203.64865273
160.0

33.0
199.945946439
170.0

48.0
199.945946439
166.592841893

33.0
194.615339375
160.0

33.0
203.473225916
170.0

33.0
202.162156143
170.0

33.0
168.960030756
160.0

48.0
177.459451847
160.0

44.0
181.213913796
159.8103599

44.0
176.09250675
169.061536341

33.0
181.213913796
180.0

44.0
174.960038609
180.0

44.0
176.400046635
185.538450468

33.0
193.846137203
213.073218491

33.0
188.416030017
179.131049422

206.487866299
180.279315936

33.0
186.944053202
185.538450468

273.0
186.569228006

33.0
176.400046635
180.0

73.5999929063
176.400046635
175.862071499

66.0
204.729432414
175.371627614

76.8529379928
204.882396457
175.564211245

36.0
259.266502807
167.264590508

59.5675646332
238.376537122
149.664974191

5

44.0
174.960038609
177.287936828

44.0
180.72005536
160.0

44.0
190.38461016
166.638134955

39.1999981091
174.81080056
175.221166877

33.0
174.81080056
166.638134955

39.1999981091
191.189198065
160.0

33.0
182.975643718
170.0

33.0
176.763032809
176.0

36.0
178.800016471
170.0

44.0
163.076880629
160.0

44.0
166.864862064
170.0

44.0
171.621631199
170.0

44.0
200.972414082
196.121023725

33.0
203.261535828
198.0

44.0
195.081951926
170.0

33.0
194.262276522
198.0

33.0
289.799984229
180.0

325.919379158
33.0
187.599987822

93.4831597336
315.0
180.0

52.4999944878
277.714992587
198.0

44.0
197.00003028
196.892301477
120.588232105

36.0
195.081951926
170.0

77.0
198.792050191
170.0

33.0
198.486477088
176.548614666

76.8529379928
198.486477088
170.0

48.0
191.217817208
130.769230652

48.0
191.699943761
97.3760323252

48.0
176.990090484
101.599990114

67.9836000803
182.538477521
103.67053805

60.0
194.754064323
98.9615263455

48.0
183.485142103
102.450888711

48.0
187.19999296
135.0

65.


36.0
195.219101693
170.0

48.0
180.108103133
160.0

48.0
167.552007981
176.30692906

36.1846122015
190.08003588
170.0

44.0
166.520256529
170.0

44.0
169.230735909
180.0

44.0
179.729731859
170.0

33.0
192.910391094
186.012310025

66.0
179.918078434
180.0

205.586233737
32.1200014024
180.0

36.0
189.0
198.0

286.0
33.0
187.0

34.7600037532
180.918015055
170.0

38.3999997646
273.0
187.0

33.0
177.459451847
170.0
31.9999945422

192.910391094
167.264590508
31.9999935099

48.0
193.999991779
150.0

44.0
176.432424078
141.517274461

44.0
163.692307603
111.419191916

64.0588376892
178.707689121
95.4000013539

60.0
159.307720522
94.9942359184

52.0
166.811878091
98.9615341073

52.0
171.680029756
109.283158644

60.0
160.0
125.940597661

60.0
181.664766195
144.0

60.0
187.586180922
150.077822817

60.0
187.704873286
158.741385747

55.0
178.378371297
159.8103599

48.0
182.926058949
169.291841694

33.0
203.623758138
214.669925152

36.4000001664
204.524737749
205.758625177

40.5999997716
203.278654

48.0
154.459462106
91.8076957483

52.0
146.217821017
94.0000014176

72.0
150.512208844
110.152285639

52.0
162.864861425
135.0

60.0
169.500005783
134.029426523

60.0
224.806128557
160.0

60.0
172.59313291
160.0

48.0
174.805499051
200.0

48.0
200.199999391
184.898454913

35.2000006833
195.081951926
180.0

37.7999991377
202.458978762
180.0

183.158666065
171.563449308

179.737000823
190.0

198.0
180.0

189.0
186.569228006

181.131067843
186.920713103

176.676901828
180.0

33.0
172.167309131
170.0

34.4769247472
186.63929526
160.0

167.552007981
160.0

44.0
142.099007628
150.208121834

48.0
139.92307987
135.0

60.0
160.0
135.0

52.0
240.0
150.0

59.9590146433
145.230761687
147.821009171

60.0
141.639325191
160.0

48.0
186.63929526
160.0

44.0
200.0
160.0

48.0
187.704873286
166.225685927

60.0
186.63929526
160.0

55.0
193.131124187
166.225685927

44.0
192.406143162
176.30692906

44.0
194.754064323
158.867209389

44.0
174.81080056
176.035044876

34.9999985039
176.400051892
176.186775792



119.0
96.1379439694
748.693607457

101.000002634
104.0
719.128380578

98.6138710094
765.928779965
104.0

96.0
104.0
753.150605178

112.0
104.0
755.060784655

98.4589941807
104.0
754.443467923

98.7805230072
104.0
714.758722623

97.8206990597
104.0
716.61890412

96.0
112.0
696.886771198

98.8235179681
104.0
733.746135394

96.0
112.0
696.103739142

112.0
109.941190983
735.00159939

112.0
152.25295321
760.0

112.0
104.0
713.574697342

112.0
104.0
779.0

112.0
104.0
741.0

96.0
94.6721208542
765.444594763

91.7647063585
83.4104037148
760.0

96.0
64.3384531702
789.791916671

75.0
98.3757336091
774.934595321

80.0
87.7294898381
788.140983188

82.4634337697
88.0551849656
741.33712493

112.0
117.0
717.220644562

99.049502178
134.290461027
704.597579032

112.0
116.548230015
679.07794937

119.0
112.0
524.615983379

160.0
129.960006699
468.792420679

119.0
112.0
471.380020189

119.0
128.46341785
463.377357706

119.0
112.0
450.940567317

119.0
112.0
465.352336827

126.0
112.0
467.018948622

126.0

...

69.2307723042
108.039047432
594.0

63.1683137962
91.0
574.852948111

90.0
106.000005475
571.828109383

137.647052021
95.788255239
556.209531864

105.0
104.0
530.801129859

112.0
96.0
428.209830886

119.0
91.0
556.773474345

112.0
96.1379439694
572.653970909

113.399972351
97.5609922847
575.511067167

112.0
95.3032709415
579.641450456

112.0
95.9016275716
591.267086193

119.0
88.7128795204
603.245634693

112.0
130.0
607.731941485

96.0
112.799975865
609.352581054

112.0
95.551733628
609.352581054

96.0
108.579317619
616.943589038

109.588239344
96.1379439694
609.520767165

105.0
91.0
615.476321185

105.0
91.0
623.501875062

105.0
91.0
624.390813641

94.6153867117
96.1379439694
588.402022484

105.0
91.0
551.181925083

96.0
91.0
621.778512273

105.0
120.0
607.653593271

97.4117700793
95.6435719875
646.525910101

108.705872541
88.0551849656
607.167153442
40.0000087461

105.0
132.0
648.0

96.0
88.7128795204
647.879805574

90.0
75.4455488299
640.503814616

91.7647063585
75.4455488299
638.723

91.7647063585
88.8688326151
640.958733711

76.7326876456
93.1172640512
608.505114861

76.9230711782
98.3641662469
653.007543762

76.9230711782
93.0000162071
625.833989939

75.0
75.19670729
658.876285793

75.0
108.89413351
621.649451988
35.0

68.9729698146
72.0000061297
674.850842593

74.04879979
84.0
656.784870419

91.8620836731
95.3032709415
644.613300071
35.8235276054

77.0000210269
104.0
988.758639986

105.0
96.1379439694
584.706593556

105.0
91.0
554.613510596

99.9508033133
116.352955252
400.633955104
42.0

111.952969451
104.0
393.486508576

96.0
136.0
391.154084093

118.588231739
136.0
394.846087983
32.1999901051

119.0
187.0
431.094337855

112.0
104.0
534.075630511
37.620004569

110.769218854
136.633681751
620.399090693
56.0

119.0
91.0
592.296950785

115.294101598
641.764963973
104.0

636.206908286
98.4589941807
91.1680221185

666.782220934
111.952969451
96.0

659.08633736
92.8536848024
88.3606399255

74.1200095406
642.688671521
87.7294898381

70.0
658.969066591
88.0551849656



89.7600246771
88.7128795204
696.979512005

90.0
614.881718655
84.0

90.0
655.376973822
72.0

75.0
64.2400155854
670.112429216

91.4594584488
64.7058798716
658.938020639

90.0
64.4230774023
663.053622972

76.9230711782
74.4117608982
619.004928351

74.3999887478
80.0482846303
732.551770858

91.4594584488
104.0
607.062455315

94.2307525824
98.3641662469
602.068995398

105.0
84.0
615.657668127

90.0
104.0
542.399893187

90.0
88.0551849656
526.500011412

100.43665639
104.0
369.243294734

100.153849431
102.687872194
358.054072514

115.640030211
112.0
394.615438465

146.369235387
112.0
347.699428614

116.812202646
112.0
372.538510909

118.938017588
104.0
382.566044724

117.869193967
104.0
490.984629947

112.0
112.307692961
548.409790434

108.777284982
95.551733628
576.215411939

112.0
589.357435505
95.551733628

112.0
607.082364378
84.0

96.0
622.970719769
88.3606399255

641.651642924
90.0
88.0551849656

76.3000091726
639.04205565
114.703468157

72.8902570013
642.032511017
75.19670729

68.972

76.6393347414
63.2432485489
630.0

76.7326876456
71.5737634322
642.147147593

70.1538489677
91.0000202103
648.0

70.5405388604
64.7058862323
607.296201267

103.040019347
113.618507087
606.389089305

90.6470571836
94.4275892066
586.75813037

72.1000007859
84.0
572.445320061

86.515265338
95.788255239
558.497233362

100.611340715
95.788255239
521.925213566

94.7161380128
91.0
370.148756447

115.050602453
104.0
370.295221683

123.939075377
122.15174531
365.904104894

146.721987913
139.189197204
361.358485536

133.058808757
116.400014688
370.295221683

102.0
168.0
380.117413885

120.615362906
104.0
589.235340345

108.259363995
112.0
581.655890842

119.0
597.042766181
96.0

111.698122273
621.9232062
96.0

632.399820277
106.506537745
88.3606399255

90.0
643.01188301
88.3606399255

68.9729698146
639.864300863
88.0551849656

82.6000170323
639.433024237
84.0

69.3600061092
641.913436323
84.0

61.0297027289
643.463971849
62.6138674328

52.0
75.4455488299
660.721666608

69.5999881344
88.712879520

109.692270652
100.983592803
565.723122838

92.8536848024
96.0
606.554679301

99.8253129315
96.829436922
610.653478831

87.8037703736
104.0
601.095315593

105.0
96.0
578.0

80.0
95.788255239
599.999947902

146.588234318
88.0551849656
587.608514451

92.3893543946
102.687872194
614.596450932

76.8292795239
91.0
582.357525383

90.0
100.52602261
600.923131001

74.5152660139
96.0
574.257989604

90.7076922673
84.0
572.938100274

101.769224715
96.0
578.0

91.7647063585
96.0
588.837141014

76.6393347414
100.52602261
591.613297582

72.4923089518
95.5294047117
629.891170801

90.0
583.655209773
107.917244808

81.9999946473
67.000004759
606.57701484

98.241253044
643.434717572
117.0

87.9999935415
75.19670729
618.339786

67.8153844886
57.8461532734
608.779245199

80.2400172277
88.3606399255
617.099994852

69.3600061092
62.8292791811
592.71075398

92.673271637
113.618507087
607.802045528

83.5135150926
84.0
557.900612949

105.599978718
93.2069057263
569.750643623

90.0
91.0
544.0

106.092694992
96.0